# PRE-PROCESSING:


In [ ]:
import json
import time
import requests
import pandas as pd
from datetime import datetime
from typing import Dict, Tuple

In [31]:
path = '../data/wyndham_smartbin_filllevel.json'

# Load
with open(path, 'r') as f:
    json_object = json.load(f)

## Isolate the data we want to use

In [32]:
rows = []
for feature in json_object["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    rows.append({
        "timestamp": props["timestamp"],
        "fullnessthreshold": props["fullnessThreshold"],
        "serialnumber": props["serialNumber"],
        "latestfullness": props["latestFullness"],
        "coordinates": coords
    })

df = pd.DataFrame(rows)

### <span style="color:green">What we have so far:</span>

In [33]:
display(df.head())

,timestamp,fullnessthreshold,serialnumber,latestfullness,coordinates
0,2018-08-13,6,1510830,0,"[144.6601949611, -37.9018111375]"
1,2018-08-13,8,1511190,4,"[144.7356233333, -37.8825216667]"
2,2018-08-13,8,1511191,0,"[144.6614092517, -37.9007616667]"
3,2018-08-13,8,1511192,0,"[144.66128, -37.900181]"
4,2018-08-13,8,1511193,4,"[144.6575503108, -37.9030024868]"


## A function that disassembles the timestamp into features we can use

<span style="color:orange">!! Several of them might be useless  !!</span>

In [34]:
def add_time_features(df, ts_col="timestamp", hemisphere="southern", tz=None):
    out = df.copy()

    # Parse to datetime
    ts = pd.to_datetime(out[ts_col])

    # Timezone handling
    if tz is not None:
        if ts.dt.tz is None:
            ts = ts.dt.tz_localize(tz)
        else:
            ts = ts.dt.tz_convert(tz)

    # ISO calendar parts (week/year/week-day)
    iso = ts.dt.isocalendar()

    # Base fields
    out["year"] = ts.dt.year
    out["quarter"] = ts.dt.quarter
    out["month"] = ts.dt.month
    out["week"] = iso.week.astype("Int64")
    out["day_of_year"] = ts.dt.dayofyear
    out["day_of_week"] = ts.dt.weekday      # 0=Mon ... 6=Sun

    # Flags
    # don't know if they are of any use
    out["is_weekend"] = out["day_of_week"].isin([5, 6])
    out["is_month_start"] = ts.dt.is_month_start
    out["is_month_end"] = ts.dt.is_month_end
    out["is_quarter_start"] = ts.dt.is_quarter_start
    out["is_quarter_end"] = ts.dt.is_quarter_end
    out["is_year_start"] = ts.dt.is_year_start
    out["is_year_end"] = ts.dt.is_year_end

    # Season mapping
    # We can argue that this is a "gerneralization" step?
    def _season(month, hemi):
        # Spring = 1, Summer = 2, Autumn = 3, Winter = 4
        if hemi == "southern":
            mapping = {12:2,1:2,2:2,
                       3:3,4:3,5:3,
                       6:4,7:4,8:4,
                       9:1,10:1,11:1}
        else:
            mapping = {12:4,1:4,2:4,
                       3:1,4:1,5:1,
                       6:2,7:2,8:2,
                       9:3,10:3,11:3}
        return mapping.get(month, pd.NA)

    out["season"] = out["month"].apply(lambda m: _season(m, hemisphere))

    return out


## Add time features and display

In [35]:
df = add_time_features(df = df, ts_col="timestamp", hemisphere="northern", tz="Europe/Madrid")
df.head()

,timestamp,fullnessthreshold,serialnumber,latestfullness,coordinates,year,quarter,month,week,day_of_year,day_of_week,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season
0,2018-08-13,6,1510830,0,"[144.6601949611, -37.9018111375]",2018,3,8,33,225,0,False,False,False,False,False,False,False,2
1,2018-08-13,8,1511190,4,"[144.7356233333, -37.8825216667]",2018,3,8,33,225,0,False,False,False,False,False,False,False,2
2,2018-08-13,8,1511191,0,"[144.6614092517, -37.9007616667]",2018,3,8,33,225,0,False,False,False,False,False,False,False,2
3,2018-08-13,8,1511192,0,"[144.66128, -37.900181]",2018,3,8,33,225,0,False,False,False,False,False,False,False,2
4,2018-08-13,8,1511193,4,"[144.6575503108, -37.9030024868]",2018,3,8,33,225,0,False,False,False,False,False,False,False,2


## Disassemble "coordinates" feature into lat and lon and add them to the dataframe

In [36]:
df["lon"] = df["coordinates"].apply(lambda x: x[0])
df["lat"] = df["coordinates"].apply(lambda x: x[1])
df.head()

,timestamp,fullnessthreshold,serialnumber,latestfullness,coordinates,year,quarter,month,week,day_of_year,...,is_weekend,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,lon,lat
0,2018-08-13,6,1510830,0,"[144.6601949611, -37.9018111375]",2018,3,8,33,225,...,False,False,False,False,False,False,False,2,144.660195,-37.901811
1,2018-08-13,8,1511190,4,"[144.7356233333, -37.8825216667]",2018,3,8,33,225,...,False,False,False,False,False,False,False,2,144.735623,-37.882522
2,2018-08-13,8,1511191,0,"[144.6614092517, -37.9007616667]",2018,3,8,33,225,...,False,False,False,False,False,False,False,2,144.661409,-37.900762
3,2018-08-13,8,1511192,0,"[144.66128, -37.900181]",2018,3,8,33,225,...,False,False,False,False,False,False,False,2,144.661280,-37.900181
4,2018-08-13,8,1511193,4,"[144.6575503108, -37.9030024868]",2018,3,8,33,225,...,False,False,False,False,False,False,False,2,144.657550,-37.903002


## Here we can see that there is something weird going on. There should be 33 entries per timestamp but thats not the case. Some timestamps have hundereds of entries. Some of them have only 32


In [37]:
timestamp_counts = df['timestamp'].value_counts()
print("Unique timestamps and their counts:")
display(timestamp_counts.head(10))

#Uncomment if you want to check how many bins there are and how many samples are present
# print("Unique Serialnumbers:")
# print(df["serialnumber"].unique())
# print(df["serialnumber"].value_counts())
# len(df["serialnumber"].unique())

Unique timestamps and their counts:


timestamp
2019-01-24    416
2018-11-29    192
2019-01-30    192
2018-12-26     64
2019-02-28     64
2020-12-23     33
2020-12-31     33
2020-12-28     33
2020-12-30     33
2020-12-29     33
Name: count, dtype: int64

## <span style="color:green">Gained Knowledge:</span> 
- there are 33 unique serialnumbers
- bin '1511202' has only 181 samples while all the others have 976 samples (drop it?)
- e.g timestamp '2019-01-24' is present 416 times. 32*16 equals 416??. So on this day the bins have been sampled 13 times?

## Next step:
Build a df that is nicer for sending requests\
This is just so the code is easier to understand\
<span style="color:orange">TODO: make sure to check if there is an API for future weather data that has the same values. Maybe create a wrapper function that combines both APIs</span>

In [38]:
# a function that returns a df which we want to use for querying the weather API
def unique_serial_coords(df):
    return (
        df[["serialnumber", "lon", "lat"]]
        .dropna(subset=["serialnumber", "lon", "lat"])
        .drop_duplicates(subset=["serialnumber"])
        .reset_index(drop=True)
    )

### Get first and last day of the dataset

In [43]:
query_timespan = (df["timestamp"].min(),df["timestamp"].max())
# should be ('2018-06-26', '2021-05-03')
print("TimeSpan:")
print(query_timespan)

TimeSpan:
('2018-06-26', '2021-05-03')


In [44]:
query_df = unique_serial_coords(df)

print("Shape of dataframe. Note: there are 33 smart-bins")
print(query_df.shape)
display(query_df.head())

Shape of dataframe. Note: there are 33 smart-bins
(33, 3)


,serialnumber,lon,lat
0,1510830,144.660195,-37.901811
1,1511190,144.735623,-37.882522
2,1511191,144.661409,-37.900762
3,1511192,144.661280,-37.900181
4,1511193,144.657550,-37.903002


## Now for every serialnumber we query the weather API from first_day to last_day

- This function takes coordinates and a timerange as input and returns a dataframe with "timestamp", "rain", "sunhours".
- It builds an URL and calls it. You can copy this to your browser and see the format in which it returns the data: 
- https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2024-01-01&end_date=2024-01-07&daily=sunshine_duration,rain_sum,precipitation_hours,daylight_duration&timezone=auto
- This example returns 1 week worth of weatherdata.

In [46]:

BASE_URL = "https://archive-api.open-meteo.com/v1/archive"

def _fetch_open_meteo_daily(lat: float, lon: float, start_date: str, end_date: str) -> pd.DataFrame:
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": ",".join([
            "sunshine_duration",   # seconds/day
            "rain_sum",            # mm/day
            "precipitation_hours", # hours/day
            "daylight_duration"    # seconds/day
        ]),
        "timezone": "auto"
    }
    r = requests.get(BASE_URL, params=params, timeout=30)
    r.raise_for_status()
    data = r.json() # Parse the JSON response

    # Create a DataFrame with the requested columns
    daily_data = {
        'timestamp': data['daily']['time'],
        'rain': data['daily']['rain_sum'],
        'sunhours': [s / 3600 for s in data['daily']['sunshine_duration']] # Convert seconds to hours
    }
    return pd.DataFrame(daily_data)


# Convert the query_df so it has one column for every timestamp. Now a cell holds the json string return value from the API


- Now we make 33 API calls. This takes a while.
- <span style="color:red"> We call the API only every 3 seconds, because otherwise they block our requests.</span>


In [ ]:
# Build one weather table per unique (serialnumber, lat, lon) and merge once
weather_frames = []
query_df = unique_serial_coords(df)  # keep full set (drop .head(1) when done testing)
# query_df = query_df.head(1) # For testing purposes
start_date, end_date = query_timespan

for _, r in query_df.iterrows():
    daily = _fetch_open_meteo_daily(r["lat"], r["lon"], start_date, end_date)  # cols: timestamp,rain,sunhours
    time.sleep(3) #0.5 seconds didnt work. 3 seconds worked
    print("Query API...")
    daily = daily.assign(
        serialnumber=r["serialnumber"],
        lat=r["lat"],
        lon=r["lon"],
    )
    weather_frames.append(daily)

weather_all = pd.concat(weather_frames, ignore_index=True)
display(weather_all.head())

Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...
Query API...


,timestamp,rain,sunhours,serialnumber,lat,lon
0,2018-06-26,0.0,6.380572,1510830.0,-37.901811,144.660195
1,2018-06-27,0.0,0.889903,1510830.0,-37.901811,144.660195
2,2018-06-28,0.0,5.696719,1510830.0,-37.901811,144.660195
3,2018-06-29,0.0,5.590622,1510830.0,-37.901811,144.660195
4,2018-06-30,3.0,4.832903,1510830.0,-37.901811,144.660195


## Now merge this dataframe into our original

In [52]:
# Merge into our main df
df = df.merge(
    weather_all[["timestamp", "serialnumber", "lat", "lon", "rain", "sunhours"]],
    on=["timestamp", "serialnumber", "lat", "lon"],
    how="left"
)

display(df.head(9))

,timestamp,fullnessthreshold,serialnumber,latestfullness,coordinates,year,quarter,month,week,day_of_year,...,is_year_end,season,lon,lat,rain_x,sunhours_x,rain_y,sunhours_y,rain,sunhours
0,2018-08-13,6,1510830,0,"[144.6601949611, -37.9018111375]",2018,3,8,33,225,...,False,2,144.660195,-37.901811,0.1,6.435839,0.1,6.435839,0.1,6.435839
1,2018-08-13,8,1511190,4,"[144.7356233333, -37.8825216667]",2018,3,8,33,225,...,False,2,144.735623,-37.882522,0.1,5.700969,0.1,5.700969,0.1,5.700969
2,2018-08-13,8,1511191,0,"[144.6614092517, -37.9007616667]",2018,3,8,33,225,...,False,2,144.661409,-37.900762,0.1,6.435839,0.1,6.435839,0.1,6.435839
3,2018-08-13,8,1511192,0,"[144.66128, -37.900181]",2018,3,8,33,225,...,False,2,144.661280,-37.900181,0.1,6.435839,0.1,6.435839,0.1,6.435839
4,2018-08-13,8,1511193,4,"[144.6575503108, -37.9030024868]",2018,3,8,33,225,...,False,2,144.657550,-37.903002,0.1,6.435839,0.1,6.435839,0.1,6.435839
5,2018-08-13,8,1511194,6,"[144.7354966667, -37.88338]",2018,3,8,33,225,...,False,2,144.735497,-37.883380,0.1,5.700969,0.1,5.700969,0.1,5.700969
6,2018-08-13,8,1511195,0,"[144.7351407368, -37.8832200529]",2018,3,8,33,225,...,False,2,144.735141,-37.883220,0.1,5.700969,0.1,5.700969,0.1,5.700969
7,2018-08-13,8,1511196,4,"[144.7351416667, -37.8841133333]",2018,3,8,33,225,...,False,2,144.735142,-37.884113,0.1,5.700969,0.1,5.700969,0.1,5.700969
8,2018-08-13,8,1511197,0,"[144.6619166667, -37.9011583333]",2018,3,8,33,225,...,False,2,144.661917,-37.901158,0.1,6.435839,0.1,6.435839,0.1,6.435839
